In [1]:
import torch
import torch.nn as nn
import torch.optim as optim  
import torchvision.transforms as transforms
import torchvision
import os
from torchvision.io import decode_jpeg
import pandas as pd
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets,models
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torchvision.io import read_image

In [2]:
PATH = './'
NUM_CLASSES = 10
BATCH_SIZE = 32

In [3]:
class CreateDataset(Dataset):
    def __init__(self, df,transform=False):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):   
        img_path = self.df.iloc[index, 0]
        image = read_image(img_path) / 255.0
        label = self.df.iloc[index, 1]

        if self.transform:
            image = self.transform(image)

        return image, label

In [4]:
transformers = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

train_dataset=CreateDataset(train_df, transformers)
test_dataset=CreateDataset(test_df, transformers)
val_dataset=CreateDataset(val_df, transformers)

In [5]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6)

In [6]:
import pytorch_lightning as pl
import torchmetrics
from torch import nn


class Model(pl.LightningModule):
    def __init__(self, output_units, dropout, learning_rate, freeze_base=False):
        super().__init__()
        self.base_model = torchvision.models.mobilenet_v2(pretrained=True)
        
        if freeze_base:
            for layer in self.base_model.features:
                for param in layer:
                    param.requires_grad = False
                
        self.base_model.classifier[0] = nn.Dropout(p=dropout)
        self.base_model.classifier[1] = torch.nn.Linear(in_features=1280, out_features=output_units)
        
        self.criterion = nn.CrossEntropyLoss()
        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()

        self.learning_rate = learning_rate
        self.save_hyperparameters()
        
    def forward(self, input_data):
        return self.base_model(input_data)

    def training_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        loss = self.criterion(preds, targets)
        self.log('train_loss', loss)
        self.train_acc(preds, targets)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=False, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        loss = self.criterion(preds, targets)
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.val_acc(preds, targets)
        self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True)
        
    def test_step(self, batch, batch_nb):
        self.validation_step(batch, batch_nb)
        
    def predict_step(self, batch, batch_nb):
        input_data, targets = batch
        preds = self(input_data)
        return torch.argmax(preds, dim=1)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

In [7]:
model = Model(NUM_CLASSES, 0.3, 1e-5)
callbacks = [
    pl.callbacks.ModelCheckpoint(monitor='val_acc', dirpath=PATH + 'Models/', verbose=True, mode='max', filename='mobile_net_t0-{val_acc:.4f}'),
    pl.callbacks.EarlyStopping(monitor='val_acc', patience=15, verbose=True, mode='max')
]

trainer = pl.Trainer(max_epochs=100, callbacks=callbacks, gpus=1)    
trainer.fit(model, train_dataloader, val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type             | Params
------------------------------------------------
0 | base_model | MobileNetV2      | 2.2 M 
1 | criterion  | CrossEntropyLoss | 0     
2 | train_acc  | Accuracy         | 0     
3 | val_acc    | Accuracy         | 0     
------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.947     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Metric val_acc improved. New best score: 0.661
Epoch 0, global step 685: val_acc reached 0.66112 (best 0.66112), saving model to "/media/ahmedessam/Storage/Codes/Machine Learning/Graduation-Project/Models/mobile_net_t0-val_acc=0.6611.ckpt" as top 1


...

Metric val_acc improved by 0.011 >= min_delta = 0.0. New best score: 0.757
Epoch 2, global step 2057: val_acc reached 0.75745 (best 0.75745), saving model to "/media/ahmedessam/Storage/Codes/Machine Learning/Graduation-Project/Models/mobile_net_t0-val_acc=0.7574.ckpt" as top 1


Validating: 0it [00:00, ?it/s]

Epoch 3, global step 2743: val_acc was not in top 1


Validating: 0it [00:00, ?it/s]

Metric val_acc improved by 0.013 >= min_delta = 0.0. New best score: 0.771
Epoch 4, global step 3429: val_acc reached 0.77062 (best 0.77062), saving model to "/media/ahmedessam/Storage/Codes/Machine Learning/Graduation-Project/Models/mobile_net_t0-val_acc=0.7706.ckpt" as top 1
/media/ahmedessam/Storage/Codes/Machine Learning/Graduation-Project/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:686: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [8]:
trainer.test(test_dataloaders=test_dataloader)

/media/ahmedessam/Storage/Codes/Machine Learning/Graduation-Project/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:902: LightningDeprecationWarning: `trainer.test(test_dataloaders)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.test(dataloaders)` instead.
  rank_zero_deprecation(
/media/ahmedessam/Storage/Codes/Machine Learning/Graduation-Project/venv/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1394: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  rank_zero_warn(
Restoring states from the checkpoint path at /media/ahmedessam/Storage/Codes/Machine Learning/Graduation-Project/Models/mobile_net_t0-val_acc=0.7706.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights fr

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_acc': 0.7658309936523438, 'val_loss': 0.7623359560966492}
--------------------------------------------------------------------------------


Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

[{'val_loss': 0.7623359560966492, 'val_acc': 0.7658309936523438}]

In [ ]:
trainer.test(test_dataloaders=train_dataloader)